# 1.  Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір). 
# Вирішіть завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

data_path = "ukr.txt"  
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

english_texts = []
ukrainian_texts = []
for line in lines[:10000]:  
    eng = line.strip().split('\t')[0]
    ukr = line.strip().split('\t')[1]
    english_texts.append(eng)
    ukrainian_texts.append('\t' + ukr + '\n')  
eng_tokenizer = Tokenizer()
ukr_tokenizer = Tokenizer(char_level=True)
eng_tokenizer.fit_on_texts(english_texts)
ukr_tokenizer.fit_on_texts(ukrainian_texts)

eng_vocab_size = len(eng_tokenizer.word_index) + 1
ukr_vocab_size = len(ukr_tokenizer.word_index) + 1

encoder_input_data = pad_sequences(eng_tokenizer.texts_to_sequences(english_texts), padding='post')
decoder_input_data = pad_sequences(ukr_tokenizer.texts_to_sequences(ukrainian_texts), padding='post')
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(eng_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(ukr_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(ukr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=10,
    validation_split=0.2
)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 109ms/step - loss: 1.3567 - val_loss: 0.7259
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 119ms/step - loss: 0.6294 - val_loss: 0.6266
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 105ms/step - loss: 0.5586 - val_loss: 0.5739
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - loss: 0.5032 - val_loss: 0.5407
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step - loss: 0.4647 - val_loss: 0.5154
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - loss: 0.4278 - val_loss: 0.4923
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 112ms/step - loss: 0.3943 - val_loss: 0.4760
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - loss: 0.3671 - val_loss: 0.4592
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - loss: 0.3358 - val_loss: 0.4429
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.3067 - val_loss: 0.4351


In [3]:
from keras.models import Model
from keras.layers import Input

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [4]:
def translate_sequence(input_seq, encoder_model, decoder_model, ukr_tokenizer):
    states = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = ukr_tokenizer.word_index['\t']  

    stop_condition = False
    decoded_sentence = ""
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ukr_tokenizer.index_word.get(sampled_token_index, '')
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > 50:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states = [h, c]

    return decoded_sentence


In [5]:
def decode_sequence(sequence, tokenizer):
    reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}
    decoded_sentence = ' '.join([reverse_word_index.get(i, '') for i in sequence if i > 0])
    return decoded_sentence

Вхідне речення (англійське): go


In [6]:
test_sentence = encoder_input_data[0:1]  
input_sentence = decode_sequence(encoder_input_data[0], eng_tokenizer)
print("Вхідне речення (англійське):", input_sentence)

# Переклад
translated = translate_sequence(test_sentence, encoder_model, decoder_model, ukr_tokenizer)
print("Перекладене речення (українське):", translated)


Вхідне речення (англійське): go
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Перекладене речення (українське): приходь!



# 2. Проведіть експерименти з моделями бібліотеки Hugging Face (раніше - Hugging Face Transformers, https://huggingface.co/) за допомогою (наприклад) Pipeline модуля

In [7]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install tf-keras --user

  Using cached tf_keras-2.18.0-py3-none-any.whl (1.7 MB)


In [12]:
from transformers import pipeline

# Завантаження класифікатора
classifier = pipeline("sentiment-analysis")

texts = [
    "I love using Hugging Face models!",
    "This is the worst experience I've ever had."
]
for text in texts:
    result = classifier(text)
    print(f"Text: {text}")
    print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.4f}\n")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\v1415\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Text: I love using Hugging Face models!
Sentiment: POSITIVE, Score: 0.9993

Text: This is the worst experience I've ever had.
Sentiment: NEGATIVE, Score: 0.9998



In [13]:
# Завантаження генеративної моделі
generator = pipeline("text-generation", model="gpt2")
seed_text = "Once upon a time"
generated_text = generator(seed_text, max_length=50, num_return_sequences=1)
print("Згенерований текст:")
for text in generated_text:
    print(text["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Згенерований текст:
Once upon a time he could hear the voices of his wife, Mrs. Lecourm, and see the bright light of his home light, yet never could he see that the voice of his own wife. That voice, too, knew nothing


In [14]:
# Завантаження моделі перекладу
translator = pipeline("translation_en_to_fr")  
text = "Hugging Face makes NLP easy to use."
translation = translator(text, max_length=50)
print("Переклад на французьку:")
print(translation[0]['translation_text'])


No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Переклад на французьку:
Hugging Face facilite l'utilisation de la NLP.


In [16]:
# Завантаження моделі NER
ner_pipeline = pipeline("ner", grouped_entities=True)
text = "Hugging Face was founded in 2016 in New York City."
entities = ner_pipeline(text)
print("Іменовані сутності:")
for entity in entities:
    print(f"Entity: {entity['entity_group']}, Text: {entity['word']}, Score: {entity['score']:.4f}")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\v1415\AppData\Roaming\P

Іменовані сутності:
Entity: ORG, Text: Hugging Face, Score: 0.9200
Entity: LOC, Text: New York City, Score: 0.9993


# 3. Завдання щодо генерації або стилізації зображень (на вибір)
# Вирішіть завдання перенесення стилю або генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE/Diffusion).

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Налаштування
image_size = 28  
batch_size = 64
timesteps = 1000  
learning_rate = 1e-4
epochs = 40

os.makedirs("fashion_mnist_results", exist_ok=True)


In [65]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) 
])

dataset = datasets.FashionMNIST(root='fashion_mnist_data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [66]:
# Лінійна шкала бета-параметрів
def linear_beta_schedule(timesteps):
    beta_start = 1e-4
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

betas = linear_beta_schedule(timesteps)
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = torch.cat([torch.tensor([1.0]), alphas_cumprod[:-1]])
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)
posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)

def add_noise(x, t, noise):
    sqrt_alpha_t = sqrt_alphas_cumprod[t].to(device)
    sqrt_one_minus_alpha_t = sqrt_one_minus_alphas_cumprod[t].to(device)
    return sqrt_alpha_t[:, None, None, None] * x + sqrt_one_minus_alpha_t[:, None, None, None] * noise


In [67]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.middle = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        return x


In [ ]:
model = UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

def save_images(images, path):
    images = (images + 1) / 2 
    save_image(images, path)

model.train()
for epoch in range(epochs):
    for batch_idx, (imgs, _) in enumerate(dataloader):
        imgs = imgs.to(device)
        batch_size = imgs.size(0)

        t = torch.randint(0, timesteps, (batch_size,), device=device).long()
        noise = torch.randn_like(imgs)
        noisy_imgs = add_noise(imgs, t, noise)

        noise_pred = model(noisy_imgs)
        loss = criterion(noise_pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Batch {batch_idx}/{len(dataloader)} Loss: {loss.item():.4f}")

    with torch.no_grad():
        test_noise = torch.randn(16, 1, image_size, image_size).to(device)
        test_images = model(test_noise)
        save_images(test_images, f"fashion_mnist_results/epoch_{epoch+1}.png")


Epoch [1/40] Batch 0/938 Loss: 0.9988
Epoch [1/40] Batch 100/938 Loss: 0.2254
Epoch [1/40] Batch 200/938 Loss: 0.1945
Epoch [1/40] Batch 300/938 Loss: 0.1781
Epoch [1/40] Batch 400/938 Loss: 0.1308
Epoch [1/40] Batch 500/938 Loss: 0.1279
Epoch [1/40] Batch 600/938 Loss: 0.1331
Epoch [1/40] Batch 700/938 Loss: 0.0778
Epoch [1/40] Batch 800/938 Loss: 0.0942
Epoch [1/40] Batch 900/938 Loss: 0.1067
Epoch [2/40] Batch 0/938 Loss: 0.1293
Epoch [2/40] Batch 100/938 Loss: 0.1125
Epoch [2/40] Batch 200/938 Loss: 0.1113
Epoch [2/40] Batch 300/938 Loss: 0.1086
Epoch [2/40] Batch 400/938 Loss: 0.0678
Epoch [2/40] Batch 500/938 Loss: 0.1123
Epoch [2/40] Batch 600/938 Loss: 0.1208
Epoch [2/40] Batch 700/938 Loss: 0.1078
Epoch [2/40] Batch 800/938 Loss: 0.1083
Epoch [2/40] Batch 900/938 Loss: 0.1189
Epoch [3/40] Batch 0/938 Loss: 0.0957
Epoch [3/40] Batch 100/938 Loss: 0.0699
Epoch [3/40] Batch 200/938 Loss: 0.1047
Epoch [3/40] Batch 300/938 Loss: 0.0870
Epoch [3/40] Batch 400/938 Loss: 0.1087
Epoch 

In [ ]:
# Генерація зображень
model.eval()
num_images = 16
generated_images = torch.randn(num_images, 1, image_size, image_size).to(device)

for t in reversed(range(timesteps)):
    z = torch.randn_like(generated_images) if t > 0 else 0
    alpha_t = alphas_cumprod[t].to(device)
    beta_t = betas[t].to(device)
    mean = (1 / torch.sqrt(alpha_t)) * (generated_images - beta_t / torch.sqrt(1 - alpha_t) * model(generated_images))
    variance = torch.sqrt(beta_t) * z
    generated_images = mean + variance

save_images(generated_images, "fashion_mnist_results/generated.png")
